# 百度网盘AI大赛-文档图片去遮挡比赛第3名方案


# 一、赛题分析
此次大赛主题结合日常生活常见情景展开，人们在使用手机等移动设备扫描证件或者扫描文档、拍摄展示资料的场景中，经常会拍摄到一些手指或者人头等其他因素，对扫描成品的美观和易用性产生了影响。期望同学们通过计算机技术对给定文档图像进行处理，帮助人们去除文档图像中的手指、人头等因素，还原真实的文档资料，提升使用效率。



# 二、 数据分析
- 本次比赛最新发布的数据集共包含训练集、A榜测试集、B榜测试集三个部分，其中训练集共14400组样本，A榜测试集共320个样本，B榜测试集共640个样本,抽取一部分数据如图：
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a.jpg)
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a_0_000.jpg)
![image](https://github.com/Jwtcode/BaiduDiskAI_DocRemoveCover_top3/blob/master/illustration/0_IMG_20220705_103123_a_0_000.png)

- hand,head 为带有手指、人头遮挡数据及遮挡部位分割图，gt 为非遮挡图片（仅有训练集数据提供gt ，A榜测试集、B榜测试集数据均不提供gt);
- annotation.txt 为图片对应关系，每一行为一组样本，用空格分隔，分别为非遮挡图片、遮挡图片、分割图片;

# 二、评价标准
评价指标为 PSNR 和 MSSSIM；

用于评价的机器环境仅提供两种框架模型运行环境：paddlepaddle 和 onnxruntime，其他框架模型可转换为
上述两种框架的模型；

机器配置：V100，显存16G，内存10G；

单张图片耗时>2s，决赛中的性能分数记0分。

由评价标准可知，不能使用大模型。

# 三、模型设计

针对图像去遮挡这个任务，我们查阅了相关资料，认为该任务是一个先擦出后重建优化的过程，所以我们选择了目前常用的EraseNet作为我们此次的baseline。整体思路分为预测目标所在位置，裁剪目标所在区域，初步去遮挡，去遮挡优化，生成mask，替换mask所在区域。
- 预测目标所在位置：
测试图片直接缩放成640x640进行预测并获得boundingbox。
预测效果如图所示：
- 裁剪目标所在区域
这里分为以下几种情况：
1）预测的boundingbox的宽高都小于1280
   目标区域向四周扩充为1280x1280。
2）预测的boundingbox的宽高有一个大于或者全都大于1280
    大于1280的边长向外扩充为64的倍数即可（避免出现维度不匹配的情况出现）
3）预测出多个boundingbox
    将多个boundingbox按面积由大到小排序，面积最大的boundingbox最先扩充，如果后面的boundingbox包含在这个扩充的boundingbox里面，则不进行处理，如果不包含其中，将其扩充为同样的尺寸，保证输入到模型中的宽高一致。
- 初步去遮挡
    改进后的EraseNet去除了预测mask的分支（训练中发现预测mask和去遮挡有冲突），整体可以看成unet结构。
- 去遮挡优化网络
    去遮挡优化网络采用U型结构的自监督学习的idr网络，通过再次的编解码进行去遮挡的二次优化，得到最终的去遮挡图。
- 生成mask
    利用yolox预测的boundbox(不扩充)在优化后的去遮挡图上裁剪出目标区域，将裁下的图片与输入图片目标所在区域做差，差值的绝对值大于等于2的地方作为mask。
- 替换mask所在区域
    将mask在原图中的位置的像素替换为mask在去遮挡图位置的像素。

![](https://ai-studio-static-online.cdn.bcebos.com/c2ced773bf7b4d9db72ed48c4b92999964dd4198103c463282fa46a62cd9d319)

其中Encoder和Decoder采用NAFBlock:

![](https://ai-studio-static-online.cdn.bcebos.com/a8b2262aaa144f8ea8c9c69087b0f93d988fc976e70e4e8ca61b2c0e88f274df)


从网络结构图上可以直观的看出EraseNet是多分支以及多阶段网络，其中包括mask生成分支和两阶段图像生成分支。此外整个网络也都是基于多尺度结构。在损失函数上，原版的ErastNet使用了感知损失以及GAN损失，这个损失函数是为了生成更加逼真的背景，但是本赛题任务的背景都是纯色，所以这两个损失是不需要的，此外EraseNet在多个尺度上使用了l1损失，我们只在第一阶段和第二阶段的最后一个尺度上使用了l1损失。我们在训练第一阶段的过程中发现预测mask的分支和第一阶段的Decoder存在冲突，所以我们去除了预测mask的分支，考虑到实效性，我们没有额外训练一个分割模型，而是选择检测模型(yolox)来获得mask。此外，根据经验，我们将EraseNet的重建网络Refinement替换为了idr网络并在底层叠加了non-local结构。



# 四、数据处理与增强

### 数据划分
1）yolox:
- 官方给的数据为14400张图片，所有图片直接缩放为640x640，其中12000张用于训练，2400张用于验证。
利用数据集自带的mask生成yolo格式标签,选用yolox作为检测模型。
2）EraseNet:
- 官方给的数据为14400张图像，我们裁剪出包含遮挡物的图片为10800张（全部为正样本，尺寸为1024x1024），其中9000张用作训练，1800张用于验证。
### 数据增广
- 增强使用横向左右翻转和小角度旋转

# 五、训练细节
- 在模型验证阶段，引入了检测网络预测的boundingbox，只将boundingbox框住的区域进行了修改，其余地方与输入保持一致，这样计算的损失只集中在包含遮挡物的区域。
- 损失函数为L1Loss

# 六、测试细节
- 原图缩放为640x640输入到yolox网络中获得boundingbox，除以缩放比例获得在原图上的boundingbox，将boundingbox所在区域扩充得到去遮挡网络的输入。
- 裁剪的图片输入到去遮挡网络中获得输出。
- 输出与输入做差获得mask.
- 将原图mask所在区域的像素值替换为输出mask所在区域的像素值。


# 七、代码结构
### code:
- config: 配置文件
- data: 定义数据增强函数
- models: 定义网络模型以及损失函数
- metrics: 定义评价指标函数
- utils: 定义其他函数
- dataset.py: 定义数据集
- output: 模型训练输出文件夹
- data_preparation.py: 训练数据裁剪脚本
- data_preparation_all.py: 全量训练数据裁剪脚本
- generate_meta_info.py: 数据筛选脚本
- generate_meta_info_all.py: 全量数据筛选脚本
- predict.py: 预测脚本
- train.py: 训练脚本
### test_code:
- predict.py: 预测脚本
- model.pdparams: 最佳权重

# 八、上分策略

![](https://ai-studio-static-online.cdn.bcebos.com/2bffa4513a1f498da0945379160fa87e420dd753765d4604bc5071e76e07de34)


上分策略主要集中在数据清洗、推理速度优化和损失函数优化:

### 数据清洗
可以看出测试数据和训练数据的domain不一致，因此采用下采样两倍后的训练数据可以极大提升网络性能。同时不必要的空白图块也会干扰网络学习到去模糊的知识，让网络误以为恢复空白图块可以恢复的很好导致收敛不充分。因此采用了数据筛选后，我们的模型性能可以进一步提升。

### 推理速度

原始代码推理速度1.67s,耗时严重，经分析，我们采用更小的模型，在精度损失不大的情况下可以获得0.83s的速度， 网络满足线上V100推理显存需求

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 

# 代码启动过程
## 训练过程

In [ ]:
### 解压数据
!cd data/ && unzip -q train_data_*.zip

: 

In [ ]:
##创建训练数据目录
! mkdir IMG
! mkdir TAR

In [ ]:
##裁剪训练数据集
import os
import cv2
import numpy as np
constprefix="train_data_"
rgb_dir='./'
IMG='./IMG'
TAR='./TAR'
#####裁剪尺寸为1024x1024
BASE=1024
for i in range(1,6):
    image_path=os.path.join(rgb_dir,constprefix + str(i))
    annotation_path=os.path.join(image_path,"annotation.txt")
    tar_filenames=[]
    inp_filenames=[]
    mask_filenames=[]
    with open (annotation_path,"r") as f:
            data=f.read().splitlines()
            for i in range (len(data)):
                cur=data[i].split(' ')
                tar_filenames.append(os.path.join(image_path,cur[0]))
                inp_filenames.append(os.path.join(image_path,cur[1]))
                mask_filenames.append(os.path.join(image_path,cur[2]))
    
    for i in range(len(mask_filenames)):
    
        mask_path=mask_filenames[i]
        inp_path=inp_filenames[i]
        tar_path=tar_filenames[i]
        filename=os.path.split(mask_path)[-1]
        mask_img=cv2.imread(mask_path,0)
        inp_img=cv2.imread(inp_path)
        tar_img=cv2.imread(tar_path)
        H,W=mask_img.shape
        mask_img[np.where(mask_img>40)]=255
        mask_img[np.where(mask_img<40)]=0
        contours, hierarchy = cv2.findContours(mask_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        x, y, w, h = cv2.boundingRect(contours[0])
        x1=x;x2=x+w;y1=y;y2=y+h;
        w_add_half=BASE//2
        h_add_half=BASE//2
        x_mid=(x1+x2)//2
        y_mid=(y1+y2)//2
        if(x_mid-w_add_half<=0):
            x1=0
            x2=BASE
        elif(x_mid+w_add_half>W):
            x2=W
            x1=W-BASE
        else:
            x1=x_mid-w_add_half
            x2=x_mid+w_add_half
        if(y_mid-h_add_half<=0):
            y1=0
            y2=BASE
            
        elif(y_mid+h_add_half>H):
            y2=H
            y1=H-BASE
            
        else:
            y1=y_mid-h_add_half
            y2=y_mid+h_add_half
        inp_filename=os.path.split(inp_path)[-1]
        tar_filename=inp_filename.split('.jpg')[0]+'_tar.jpg'
        new_inp_path=os.path.join(IMG,inp_filename)
        new_tar_path=os.path.join(TAR,tar_filename)
        new_inp_img=inp_img[y1:y2,x1:x2,:]
        new_tar_img=tar_img[y1:y2,x1:x2,:]
        cv2.imwrite(new_inp_path,new_inp_img)
        cv2.imwrite(new_tar_path,new_tar_img)

In [ ]:
## 去除不必要的zip文件
! rm -rf *.zip

: 

In [ ]:
## 安装训练所需的python包
! pip install torch==1.7.1
! pip install scikit-image
! pip install warmup_scheduler
! pip install pillow

: 

In [ ]:
## 利用L1Loss训练
! cd ../ && python train.py 

: 

## 预测过程
按照官方给定的预测脚本运行方式相同，将最终的finetuning模型改名为model.pdparams，并且放在predict.py的同意目录下，使用命令：
```
python predict.py [src_image_dir] [results]
```

In [ ]:
## 测试脚本
! cd test_code/ && python predict.py {your_test_data_path} {save_path}

: 